In [71]:
import pandas as pd
from pymongo import MongoClient
from textblob import TextBlob

# Connect to MongoDB (replace 'your_connection_string' and 'your_database' with your actual values)
client = MongoClient('mongodb://localhost:27017')
db = client['nytimes']
collection = db['comment months']

# Fetch all documents from the collection
cursor = collection.find()

# Convert MongoDB cursor to a list of dictionaries
data_list = list(cursor)


df = pd.DataFrame(data_list)



happy = 0
sad = 0
neutral = 0

def analyze_sentiment(comment):
    global happy, sad, neutral  # Declare the variables as global
    if not isinstance(comment, str):
        return "Not a string"
    # Create a TextBlob object
    blob = TextBlob(comment)

    # Get the sentiment polarity
    sentiment_polarity = blob.sentiment.polarity

    # Classify the sentiment based on polarity
    if sentiment_polarity > 0:
        #happy = happy + 1
        return "happy"
    elif sentiment_polarity < 0:
        #sad = sad + 1
        return "sad"
    else:
        #neutral = neutral + 1
        return "neutral"

month_map = {}

# Assuming df is your DataFrame with a column 'commentBody'
count = 0
for index, row in df.iterrows():
    comment = row['commentBody']
    date = row['Date']

    if date not in month_map:
        month_map[date] = {
            "happy": 0,
            "sad": 0,
            "neutral": 0
            }
    else:
        result = analyze_sentiment(comment)

    #print(result)

    if result == "Not a string":
        continue
    elif result == "happy":
        month_map[date]["happy"] += 1
    elif result == "sad":
        month_map[date]["sad"] += 1
    else:
        month_map[date]["neutral"] += 1
    #else:
    #    month_map[date][result] += 1

    
#print("Happy:", happy)
#print("Sad:", sad)
#print("Neutral:",neutral)

month_map

db = client["nytimes"]

# Create a collection (table) in the database
collection = db["monthcomments"]

for month, values in month_map.items():
    document = {
        "month": month,
        "happy": values["happy"],
        "sad": values["sad"],
        "neutral": values["neutral"]
    }
    collection.insert_one(document)
# Disconnect from MongoDB
client.close()  
